In [5]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
    .appName("Binance - Hyper price increase detection")
    .getOrCreate())

In [6]:
ohlcvDF = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "localhost:9092").option("subscribe", "binance_data").option("startingOffsets", "earliest").load()



In [7]:
from pyspark.sql.functions import split, col
from pyspark.sql.types import DoubleType, LongType

#TCBUSD,1648234367780,43974.77000000

ohlcvDF = ohlcvDF.selectExpr("CAST(value AS STRING)").select(split("value",'\,').alias("fields")).withColumn("timestamp",col("fields").getItem(1).cast(LongType())).withColumn("symbol",col("fields").getItem(0)).withColumn("open",col("fields").getItem(2).cast(DoubleType())).drop("fields")


In [8]:
# from pyspark.sql.functions import concat, lit

# ohlcvDF.writeStream \
#         .format("console") \
#         .start() \
#         .awaitTermination()

In [9]:
from pyspark.sql import Row
from datetime import datetime
from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

#https://docs.influxdata.com/influxdb/cloud/api-guide/client-libraries/python/
class InfluxDBWriter:
    def __init__(self):
        self.t = 'C3GlPOdiT0dqAH9LH_Q8Z0vtXdkZ84DAkFE0phwEtgR_lbc7TYP4Sz6rTavhbNcLIjT9joY15Lyz4x1kkd30LA=='
        self.o = 'project' # database -> org
        self.b = 'binance' # table -> bucket
        self.client = InfluxDBClient(url="http://localhost:8086", token=self.t, org=self.o) # client creation
        self.write_api = self.client.write_api() # influx db write api

    def open(self, partition_id, epoch_id): # opens the connection to influx db
        print("Opened %d, %d" % (partition_id, epoch_id))
        return True

    def process(self, row): # writing the data into influx db table
        self.write_api.write(bucket=self.b, record=self._row_to_line_protocol(row))

    def close(self, error): #closing write apis and influxdb client
        self.write_api.__del__()
        self.client.__del__()
        print("Closed with error: %s" % str(error))

    def _row_to_line_protocol(self, row: Row): #converting the data into the format how it is stored in influx db
        # TODO map Row to LineProtocol
        p = Point("mess").tag("coin", row['symbol']).field("value", row['open']).time(row['timestamp'], WritePrecision.MS)
        return p
    
ohlcvDF.writeStream.foreach(InfluxDBWriter()).start().awaitTermination()
# we are taking the data and writing it into influx db using write stream API of spark



22/03/29 08:52:57 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/np/7wstqrjj7h77xmr2lv_bt64h0000gn/T/temporary-96d88a35-4262-4da6-b625-9cea03df79ee. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
22/03/29 08:52:57 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 140
-------------------------------------------
+-------------+-------+--------+
|    timestamp| symbol|    open|
+-------------+-------+--------+
|1648536777273|BTCBUSD|46878.73|
+-------------+-------+--------+



-------------------------------------------
Batch: 141
-------------------------------------------
+-------------+-------+--------+
|    timestamp| symbol|    open|
+-------------+-------+--------+
|1648536778200|BTCBUSD|46878.74|
+-------------+-------+--------+



Opened 0, 0                                                                     


-------------------------------------------
Batch: 142
-------------------------------------------
+-------------+-------+--------+
|    timestamp| symbol|    open|
+-------------+-------+--------+
|1648536779292|BTCBUSD|46878.73|
+-------------+-------+--------+



...

-------------------------------------------
Batch: 1891
-------------------------------------------
+-------------+-------+--------+
|    timestamp| symbol|    open|
+-------------+-------+--------+
|1648538662567|BTCBUSD|46991.08|
+-------------+-------+--------+



Opened 0, 1749
Closed with error: None


-------------------------------------------
Batch: 1892
-------------------------------------------
+-------------+-------+--------+
|    timestamp| symbol|    open|
+-------------+-------+--------+
|1648538663569|BTCBUSD|46987.22|
+-------------+-------+--------+



Opened 0, 1750
Closed with error: None
22/03/29 09:26:36 ERROR Utils: Aborting taskge 3645:>               (0 + 1) / 1]
org.apache.kafka.common.errors.TimeoutException: Timeout of 60000ms expired before the position for partition binance_data-0 could be determined
22/03/29 09:26:36 ERROR Utils: Aborting task
org.apache.kafka.common.errors.TimeoutException: Timeout of 60000ms expired before the position for partition binance_data-0 could be determined
22/03/29 09:26:36 ERROR DataWritingSparkTask: Aborting commit for partition 0 (task 3644, attempt 0, stage 3644.0)
22/03/29 09:26:36 ERROR DataWritingSparkTask: Aborting commit for partition 0 (task 3645, attempt 0, stage 3645.0)
22/03/29 09:26:36 ERROR DataWritingSparkTask: Aborted commit for partition 0 (task 3644, attempt 0, stage 3644.0)
22/03/29 09:26:36 ERROR DataWritingSparkTask: Aborted commit for partition 0 (task 3645, attempt 0, stage 3645.0)
Opened 0, 1751
Closed with error: None
22/03/29 09:26:36 ERROR Executor: Exception in t

StreamingQueryException: Writing job aborted
=== Streaming Query ===
Identifier: [id = e5054cf8-9247-48cc-b926-812a47940eef, runId = 8dbf611b-f2b2-40f5-844e-80336473f129]
Current Committed Offsets: {KafkaV2[Subscribe[binance_data]]: {"binance_data":{"0":9867}}}
Current Available Offsets: {KafkaV2[Subscribe[binance_data]]: {"binance_data":{"0":9868}}}

Current State: ACTIVE
Thread State: RUNNABLE

Logical Plan:
WriteToMicroBatchDataSource org.apache.spark.sql.execution.streaming.sources.ForeachWrite$$anon$2@4b00c2a7
+- Project [timestamp#2868L, symbol#2871, open#2876]
   +- Project [fields#2866, timestamp#2868L, symbol#2871, cast(fields#2866[2] as double) AS open#2876]
      +- Project [fields#2866, timestamp#2868L, fields#2866[0] AS symbol#2871]
         +- Project [fields#2866, cast(fields#2866[1] as bigint) AS timestamp#2868L]
            +- Project [split(value#2864, \,, -1) AS fields#2866]
               +- Project [cast(value#2851 as string) AS value#2864]
                  +- Str